# 1. Setup and Library Installation

In this section, we install the necessary libraries (fastbook, waterfallcharts, treeinterpreter, dtreeviz) and import key modules for tabular data analysis and machine learning.

In [ ]:
#hide
! pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz==1.4.1
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

# 2. Data Loading and Preprocessing

Here, we load the dataset, parse dates, and define our dependent variable (`5m_forward_returns`). We also set up the `TabularPandas` object from the `fastai` library, which handles splitting the data into training and validation sets (based on time) and preprocessing steps like filling missing values and categorifying variables.

In [ ]:
df = pd.read_csv("data/data.csv",
                 low_memory=False,
                 parse_dates=["timestamp", "minute_utc"],
                 )
df["minute_utc"] = pd.to_datetime(df["minute_utc"], utc=True, errors="coerce")

In [ ]:
df.columns

In [ ]:
dep_var = "5m_forward_returns"

In [ ]:
procs = [FillMissing]

In [ ]:
cond1 = (df.index < 58960) & df['5m_forward_returns'].notna()
cond2 = (df.index > 58964) & df['5m_forward_returns'].notna()
train_idx = np.where(cond1)[0]
valid_idx = np.where(cond2)[0]

splits = (list(train_idx), list(valid_idx))

In [ ]:
cont, cat = cont_cat_split(df, 1, dep_var=dep_var)

In [ ]:
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

In [ ]:
len(to.train), len(to.valid)

In [ ]:
to.show(5)

In [ ]:
save_pickle('data/to.pkl', to)

In [ ]:
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

# 3. Decision Tree Baseline

We start by training a simple Decision Tree Regressor. This serves two purposes:
1. **Visualization**: We train a small tree to visualize the decision logic.
2. **Baseline Performance**: We train a larger tree to check for overfitting and establish a baseline Root Mean Squared Error (RMSE) to beat.

In [ ]:
xs_cleaned = xs.drop(columns=['timestamp', 'minute_utc'])
valid_xs_cleaned = valid_xs.drop(columns=['timestamp', 'minute_utc'])
m = DecisionTreeRegressor(max_leaf_nodes = 4)
m.fit(xs_cleaned, y);

In [ ]:
draw_tree(m, xs_cleaned, size=10, leaves_parallel=True, precision=2)

In [ ]:
samp_idx = np.random.permutation(len(y))[:500]
dtreeviz(m, xs_cleaned.iloc[samp_idx], y.iloc[samp_idx], xs_cleaned.columns, dep_var,
        fontname='DejaVu Sans', scale=1.6, label_fontsize=10,
        orientation='TD')

In [ ]:
m = DecisionTreeRegressor()
m.fit(xs_cleaned, y);

In [ ]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

In [ ]:
m_rmse(m, xs_cleaned, y)

In [ ]:
m_rmse(m, valid_xs_cleaned, valid_y)

In [ ]:
m.get_n_leaves(), len(xs_cleaned)

In [ ]:
m = DecisionTreeRegressor(min_samples_leaf=25)
m.fit(xs_cleaned, y)
m_rmse(m, xs_cleaned, y), m_rmse(m, valid_xs_cleaned, valid_y)

In [ ]:
m.get_n_leaves()

# 4. Random Forest Model

We transition to a Random Forest Regressor to improve generalization. This section includes:
- Defining a Random Forest function.
- Training the model.
- analyzing the RMSE on the validation set.
- Checking how the error converges as we add more trees (estimators).

In [ ]:
def rf(xs, y, n_estimators=200, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators,
                                 min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [ ]:
m = rf(xs_cleaned, y);

In [ ]:
m_rmse(m, xs_cleaned, y), m_rmse(m, valid_xs_cleaned, valid_y)

In [ ]:
y_std = np.std(valid_y)
print(y_std)

In [ ]:
preds = np.stack([t.predict(valid_xs_cleaned) for t in m.estimators_])
plt.plot([r_mse(preds[:i+1].mean(0), valid_y) for i in range(1000)]);

In [ ]:
r_mse(m.oob_prediction_, y)

In [ ]:
preds.shape

In [ ]:
preds_std = preds.std(0)

In [ ]:
preds_std[:5]

# 5. Feature Importance and Selection

Using the Random Forest model, we calculate feature importance to identify the most predictive signals. We then:
- Filter out low-importance features.
- Analyze redundant features (using clustering and Out-of-Bag scores) to create a more compact and efficient feature set (`xs_final`).

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [ ]:
fi = rf_feat_importance(m, xs_cleaned)
fi[:10]

In [ ]:
def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

plot_fi(fi[:30])

In [ ]:
to_keep = fi[fi.imp>0.015].cols
len(to_keep)

In [ ]:
xs_imp = xs_cleaned[to_keep]
valid_xs_imp = valid_xs_cleaned[to_keep]

In [ ]:
m = rf(xs_imp, y)

In [ ]:
m_rmse(m, xs_imp, y), m_rmse(m, valid_xs_imp, valid_y)

In [ ]:
len(xs_cleaned.columns), len(xs_imp.columns)

In [ ]:
fi = rf_feat_importance(m, xs_imp)
plot_fi(fi);

In [ ]:
cluster_columns(xs_imp)

In [ ]:
def get_oob(df):
    m = RandomForestRegressor(n_estimators=200, min_samples_leaf=15,
                              max_features=0.5, n_jobs=-1, oob_score=True)
    m.fit(df, y)
    return m.oob_score_

In [ ]:
get_oob(xs_imp)

In [ ]:
{c:get_oob(xs_imp.drop(c, axis=1)) for c in (
    'realized_vol_60', 'realized_vol', 'STOCH', 'MOM_10',
    'VWAP_res', 'RSI_slow', 'RSI_fast', 'KAMA_res',
    'MOM_5', 'MACD_hist', 'ADOSC_lag_2', 'ADOSC_lag_3',
    'ADX', 'sin_time_cycle')}

In [ ]:
to_drop = ['realized_vol_60', 'realized_vol','RSI_fast', 'KAMA_res', 'VWAP_res', 'RSI_slow',
           'ADOSC_lag_2', 'ADOSC_lag_3', 'ADX', 'sin_time_cycle']
get_oob(xs_imp.drop(to_drop, axis=1))

In [ ]:
xs_final = xs_imp.drop(to_drop, axis=1)
valid_xs_final = valid_xs_imp.drop(to_drop, axis=1)

In [ ]:
save_pickle('data/xs_final.pkl', xs_final)
save_pickle('data/valid_xs_final.pkl', valid_xs_final)

In [ ]:
m = rf(xs_final, y)
m_rmse(m, xs_final, y), m_rmse(m, valid_xs_final, valid_y)

In [ ]:
ax = valid_xs_final['STOCH'].hist()

In [ ]:
ax = valid_xs_final['MOM_10'].hist()

In [ ]:
ax = valid_xs_final['RSI'].hist()

In [ ]:
ax = valid_xs_final['KAMA_res'].hist()

# 6. Model Interpretation

To understand *how* the model makes predictions, we use:
- **Partial Dependence Plots**: To see the marginal effect of specific features.
- **Waterfall Charts**: To break down a specific prediction instance contribution by contribution.
- **Domain Shift Check**: We also check if the model can distinguish between training and validation sets, which would indicate a drift in data distribution.

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 4))

PartialDependenceDisplay.from_estimator(
    m,                   # your fitted model
    valid_xs_final,      # your validation features (DataFrame or array)
    ['STOCH', 'MOM_10'],  # features
    grid_resolution=20,
    ax=ax
)
plt.tight_layout()

In [ ]:
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 4))

PartialDependenceDisplay.from_estimator(
    m,                   # your fitted model
    valid_xs_final,      # your validation features (DataFrame or array)
    ['RSI', 'KAMA_res'],  # features
    grid_resolution=20,
    ax=ax
)
plt.tight_layout()

In [ ]:
#hide
import warnings
warnings.simplefilter('ignore', FutureWarning)

from treeinterpreter import treeinterpreter
from waterfall_chart import plot as waterfall

In [ ]:
row = valid_xs_final.iloc[:5]

In [ ]:
prediction,bias,contributions = treeinterpreter.predict(m, row.values)

In [ ]:
prediction[0], bias[0], contributions[0].sum()

In [ ]:
waterfall(valid_xs_final.columns, contributions[0], threshold=0.08,
          rotation_value=45,formatting='{:,.8f}');

In [ ]:
df_dom = pd.concat([xs_final, valid_xs_final])
is_valid = np.array([0]*len(xs_final) + [1]*len(valid_xs_final))

m = rf(df_dom, is_valid)
rf_feat_importance(m, df_dom)[:6]

In [ ]:
m = rf(xs_final, y)
print('orig', m_rmse(m, valid_xs_final, valid_y))

c = 'bandwidth_5'
m = rf(xs_final.drop(c,axis=1), y)
print(c, m_rmse(m, valid_xs_final.drop(c,axis=1), valid_y))

# 7. Neural Network Model

We train a Neural Network using `fastai`'s tabular learner to see if a deep learning approach captures different patterns than the tree-based models. We then create an ensemble by averaging the Neural Network predictions with the Random Forest predictions.

In [ ]:
df_nn = pd.read_csv("data/data.csv",
                 low_memory=False,
                 parse_dates=["timestamp", "minute_utc"],
                 )
df_nn["minute_utc"] = pd.to_datetime(df_nn["minute_utc"], utc=True, errors="coerce")

In [ ]:
df_nn_final = df_nn[list(xs_final.columns) + [dep_var]].copy()
df_nn_final.dropna(subset=[dep_var], inplace=True)

In [ ]:
cont_nn,cat_nn = cont_cat_split(df_nn_final, max_card=40, dep_var=dep_var)

In [ ]:
cont_nn, cat_nn

In [ ]:
procs_nn = [Categorify, FillMissing, Normalize]

# Recalculate splits based on the cleaned df_nn_final indices
current_indices = df_nn_final.index
new_train_indices = current_indices[current_indices < 58960].tolist()
new_valid_indices = current_indices[current_indices > 58964].tolist()
updated_splits = (new_train_indices, new_valid_indices)

to_nn = TabularPandas(df_nn_final, procs_nn, cat_nn, cont_nn,
                      splits=updated_splits, y_names=dep_var)

In [ ]:
print(f"Unique values in 'volume': {df_nn_final['AROON'].nunique()}")
print(f"Data type of 'volume': {df_nn_final['AROON'].dtype}")

In [ ]:
dls = to_nn.dataloaders(1024)

In [ ]:
y = to_nn.train.y
y.min(),y.max()

In [ ]:
learn = tabular_learner(dls, y_range=(-7.5,5), layers=[500, 250],
                        n_out=1, loss_func=F.mse_loss,
                        config=tabular_config(ps=[0.25, 0.25], embed_p=0.25))

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(50, 5e-4)

In [ ]:
preds,targs = learn.get_preds()
r_mse(preds,targs)

In [ ]:
learn.save('nn')

In [ ]:
m = rf(xs_final, y)
rf_preds = m.predict(to_nn.valid.xs[xs_final.columns])
ens_preds = (to_np(preds.squeeze()) + rf_preds) /2

In [ ]:
r_mse(ens_preds, to_nn.valid.y.values)

# 8. XGBoost Model with Hyperparameter Tuning

Finally, we implement an XGBoost model, known for its high performance on tabular data. We use **Optuna** to perform Bayesian optimization and find the best hyperparameters (learning rate, depth, regularization, etc.) to minimize RMSE on the validation set. We then retrain the best model and perform a detailed analysis of its residuals and feature correlations.

In [ ]:
!pip install xgboost
!pip install optuna

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost.callback import EarlyStopping

In [ ]:
# 1. Baseline RMSE (Guessing the mean)
baseline_preds = np.ones(len(valid_y)) * valid_y.mean()
baseline_rmse = r_mse(baseline_preds, valid_y)

# 2. Random Forest RMSE (Ensure we use the correct unnormalized validation set)
# Note: 'm' is the trained Random Forest from previous cells
rf_preds_corrected = m.predict(valid_xs_final)
rf_rmse = r_mse(rf_preds_corrected, valid_y)

# 3. Ensemble RMSE
# 'preds' contains the NN predictions from learn.get_preds()
# We average the corrected RF preds and the NN preds
# We assume 'preds' aligns with 'valid_y' (both from validation set)
ens_preds_corrected = (to_np(preds.squeeze()) + rf_preds_corrected) / 2
ens_rmse = r_mse(ens_preds_corrected, valid_y)

# Calculate Edge
rf_edge = baseline_rmse - rf_rmse
ens_edge = baseline_rmse - ens_rmse

print(f"Baseline RMSE: {baseline_rmse:.6f}")
print(f"Random Forest RMSE: {rf_rmse:.6f} (Edge: {rf_edge:.6f})")
print(f"Ensemble RMSE: {ens_rmse:.6f} (Edge: {ens_edge:.6f})")

In [ ]:
import numpy as np
import optuna


# Ensure no NaNs/Infs in target
# Train set cleaning
mask_train = ~y.isna() & ~np.isinf(y)
if (~mask_train).any():
    print(f"Dropping {(~mask_train).sum()} rows from train with NaN/Inf targets")
    xs_final = xs_final[mask_train]
    y = y[mask_train]

# Validation set cleaning
mask_valid = ~valid_y.isna() & ~np.isinf(valid_y)
if (~mask_valid).any():
    print(f"Dropping {(~mask_valid).sum()} rows from valid with NaN/Inf targets")
    valid_xs_final = valid_xs_final[mask_valid]
    valid_y = valid_y[mask_valid]

def objective(trial):
    params = {
        # Keep this high and let early stopping choose effective complexity
        "n_estimators": 6000,
        "early_stopping_rounds": 50,  # Moved here: parameter for constructor

        # Core knobs
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-2, 20.0, log=True),
        "gamma": trial.suggest_float("gamma", 0.0, 10.0),  # min_split_loss

        # Sampling (regularization via randomness)
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),

        # L1/L2 regularization
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-9, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-6, 100.0, log=True),

        # Plumbing
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "tree_method": "hist",
        "random_state": 42,
        "n_jobs": -1,
        "verbosity": 0,
    }

    model = XGBRegressor(**params)

    model.fit(
        xs_final,
        y,
        eval_set=[(valid_xs_final, valid_y)],
        # early_stopping_rounds removed from here
        verbose=False,
    )

    preds = model.predict(valid_xs_final)
    score = r_mse(valid_y, preds)

    # Useful for debugging / analyzing runs
    if getattr(model, "best_iteration", None) is not None:
        trial.set_user_attr("best_iteration", int(model.best_iteration))

    return score

study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=7),
)

study.optimize(objective, n_trials=100, show_progress_bar=True)

print("Best RMSE:", study.best_value)
print("Best params:", study.best_params)
print("Best best_iteration:", study.best_trial.user_attrs.get("best_iteration"))

In [ ]:
# Calculate XGBoost Edge
best_xgb_rmse = study.best_value
xgb_edge = baseline_rmse - best_xgb_rmse

print(f"Baseline RMSE: {baseline_rmse:.6f}")
print(f"Best XGBoost RMSE: {best_xgb_rmse:.6f}")
print(f"XGBoost Edge: {xgb_edge:.6f}")

## Retrain Best XGBoost

Retrain the XGBoost model on the training set using the best hyperparameters identified by the Optuna study to establish a baseline for analysis.


In [ ]:
best_params = study.best_params.copy()

# Add fixed params
best_params.update({
    "n_estimators": 6000,
    "early_stopping_rounds": 50,
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "tree_method": "hist",
    "random_state": 42,
    "n_jobs": -1,
    "verbosity": 0,
})

model = XGBRegressor(**best_params)

model.fit(
    xs_final,
    y,
    eval_set=[(valid_xs_final, valid_y)],
    verbose=False,
)

preds_xgb = model.predict(valid_xs_final)
rmse = r_mse(preds_xgb, valid_y)
print(f"Retrained XGBoost RMSE: {rmse}")

## Analyze Feature Importance

Generate and visualize a feature importance plot (using gain) from the retrained XGBoost model to identify the most influential predictors.


In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt

# Plot feature importance
fig, ax = plt.subplots(figsize=(10, 8))
plot_importance(model, importance_type='gain', max_num_features=20, height=0.5, ax=ax)
plt.show()

# Extract and display feature importance as DataFrame
importance = model.get_booster().get_score(importance_type='gain')
importance_df = pd.DataFrame(list(importance.items()), columns=['Feature', 'Gain'])
importance_df = importance_df.sort_values(by='Gain', ascending=False)
print(importance_df.head(10))

## Analyze Feature Correlations

Compute and plot a correlation heatmap between the top influential features and the target variable to detect linear relationships and redundancy.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Extract top 15 feature names
top_features = importance_df['Feature'].head(15).tolist()

# 2. Create temporary DataFrame with top features and target
# We use .copy() to avoid SettingWithCopy warnings if xs_final is a view
temp_df = xs_final[top_features].copy()
temp_df['target'] = y

# 3. Calculate correlation matrix
corr_matrix = temp_df.corr()

# 4. Visualize correlation matrix using a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title("Correlation Heatmap: Top 15 Features vs Target")
plt.show()

## Analyze Target Autocorrelation

Plot the Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) of the target variable to assess the potential value of adding lag-based features.


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

# Create figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot ACF
plot_acf(y, lags=50, zero=False, ax=axes[0], title='Autocorrelation Function (ACF)')

# Plot PACF
plot_pacf(y, lags=50, zero=False, ax=axes[1], title='Partial Autocorrelation Function (PACF)')

plt.tight_layout()
plt.show()

## Analyze Model Residuals

Calculate residuals on the validation set and visualize them to identify patterns.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Calculate residuals
residuals = valid_y - preds_xgb

# 2. Create figure with 3 subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 3. Residuals vs Predicted Values
axes[0].scatter(preds_xgb, residuals, alpha=0.5, s=10)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_xlabel('Predicted Values')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residuals vs. Predicted Values')

# 4. Residuals vs Time (Index)
axes[1].scatter(valid_y.index, residuals, alpha=0.5, s=10)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Index (Time)')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residuals vs. Time')

# 5. Histogram of Residuals
sns.histplot(residuals, kde=True, bins=50, ax=axes[2])
axes[2].set_xlabel('Residuals')
axes[2].set_title('Distribution of Residuals')

# 6. Display the figure
plt.tight_layout()
plt.show()

# Optional: Print summary statistics
print("Residuals Statistics:")
print(residuals.describe())

## Walkforward Validation

Use `TimeSeriesSplit` to verify edge of best XGBoost model on test set.



In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# 4. Walk-Forward Validation Loop
tscv = TimeSeriesSplit(n_splits=5)

params_wfv = study.best_params.copy()
params_wfv.update({
    "n_estimators": 2000, # Reduced slightly for speed during WFV loop
    "early_stopping_rounds": 50,
    "tree_method": "hist",
    "n_jobs": -1,
    "verbosity": 0
})

rmse_scores = []

print("Starting Walk-Forward Validation...")

for fold, (train_index, test_index) in enumerate(tscv.split(xs_final)):
    # Split data
    X_train, X_test = xs_final.iloc[train_index], xs_final.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Init and Fit
    model_wfv = XGBRegressor(**params_wfv)
    model_wfv.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False
    )

    # Predict
    preds = model_wfv.predict(X_test)
    score = r_mse(preds, y_test)
    rmse_scores.append(score)

    print(f"Fold {fold+1} RMSE: {score:.6f} (Train Size: {len(X_train)}, Test Size: {len(X_test)})")

print(f"\nAverage WFV RMSE: {np.mean(rmse_scores):.6f}")
print(f"Std Dev WFV RMSE: {np.std(rmse_scores):.6f}")